In [22]:
import h5py
import json
import random
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback
import pydicom
sys.path.append('./src')
import numpy as np
#from generators import DataGenerator
from generator_hamdi import DataGenerator
from models import dota_energies
from preprocessing import DataRescaler
from preprocessing_hamdi import get_scaling_factors
from tensorflow_addons.optimizers import LAMB
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.config import list_physical_devices
import json
import tensorflow as tf
print(list_physical_devices('GPU'))
from models import dota_energies
from preprocessing import DataRescaler
from evaluation import gamma_analysis, error_analysis


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


In [4]:
import os
import pandas as pd

folder_path = r"/tudelft.net/staff-umbrella/simelectrons/OneGeometryMultipleEnergy"  # Replace with the actual folder path

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Initialize an empty list to store the dataframes
dfs = []

# Loop through each file in the folder
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file is a pickle file
    if file_name.endswith('.pkl'):
        # Load the dataframe from the pickle file
        df = pd.read_pickle(file_path)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
concatenated_df = pd.concat(dfs, ignore_index=True)


In [6]:
concatenated_df.to_pickle(os.path.join(folder_path, 'All_Simulation_Data.pkl'))

In [11]:
df=pd.read_pickle(os.path.join(folder_path, 'All_Simulation_Data.pkl'))
df

,text_file_names,dose_file_names,energy,beam coordinates,angles,ct_name,cropped_dose_name,cropped_geometry_name,shape
0,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0000,353.325404,"(18.3, -6.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,1d4bd3c5-7c51-44bb-b84a-7164a2e57e3a,c05dcc72-d5d6-445d-a1df-cf5f3eb27343,"(68, 68, 150)"
1,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0001,374.795800,"(18.3, -5.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,64cb414b-cff6-469a-83b2-4c1b333d70e7,4588e18d-33f2-4f2a-9b69-fdcfdc089212,"(68, 68, 150)"
2,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0002,191.654047,"(18.3, -4.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,7ea270b7-9f67-4f34-9280-ca53ab2c0372,4fd9379b-0ba5-4cb1-b6fe-1ef9c40a6796,"(68, 68, 150)"
3,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0003,358.677510,"(18.3, -3.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,7e67d860-8861-4766-b82c-ef4b864165ec,13241077-f950-4ec9-8ad2-0b4b5e91a744,"(68, 68, 150)"
4,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0004,191.364991,"(18.3, -2.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,010411fc-202b-4448-b05e-c35e80929feb,d65e1be8-79d1-4477-bbc8-3f1a7d043cfb,"(68, 68, 150)"
...,...,...,...,...,...,...,...,...,...
34315,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2523,127.346121,"(5.2, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,f8450e55-d32a-43b2-9edd-7ab1f7919acb,6d0fbeba-c538-45c3-b605-aed2ba309966,"(68, 68, 150)"
34316,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2524,154.130103,"(6.2, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,78668a49-c7bb-4051-a245-a8609237a96a,5e48857b-cd3c-4458-a56f-b7f4fbdc4de2,"(68, 68, 150)"
34317,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2525,137.009227,"(7.2, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,efdc242c-33ea-475d-9332-bdbcaf0b3937,0cf6baf5-54eb-41c3-8111-f0dba1979d2f,"(68, 68, 150)"
34318,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2526,355.500052,"(8.2, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,0d8b364e-7148-4bc9-8826-289cf6ea0d15,bdf8499e-364e-4da6-8fb9-8ebfc89500f4,"(68, 68, 150)"


In [13]:
filtered_df = df[df['shape'] == (68, 68,150)]
filtered_df.to_pickle(os.path.join(folder_path, 'Filtered_Simulation_Data.pkl'))


In [18]:
gpu_index = 0

# Set the GPU to be visible and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print('Available GPUs: ', gpus)
mem_growth = tf.config.experimental.get_memory_growth(gpus[gpu_index])
print('Memory growth: ', mem_growth)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[gpu_index], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[gpu_index], True)
    mem_growth = tf.config.experimental.get_memory_growth(gpus[gpu_index])
    print('GPU set to be visible and memory growth set to: ', mem_growth)

'''Initializing Batch size num epochs learning rate and weight decay. This is the same in my model'''
batch_size =  8
num_epochs = 30
learning_rate = 0.001
weight_decay = 0.0001

with open('./hyperparam.json', 'r') as hfile:
    param = json.load(hfile)

preprocess=True
# Load data files
path = r"/tudelft.net/staff-umbrella/simelectrons/OneGeometryMultipleEnergy"
data_df=pd.read_pickle(os.path.join(path,'Filtered_Simulation_Data.pkl'))
path_ckpt = './weights/ckpt_multiple_energy/weights.ckpt'
train_split = 0.7
val_split = 0.15
test_split = 0.15
listIDs=data_df['cropped_geometry_name'].tolist()
random.seed(333)
random.shuffle(listIDs)


trainIDs = listIDs[:int(round(train_split*len(listIDs)))]
valIDs = listIDs[int(round(train_split*len(listIDs))):]
testIDs = listIDs[int(round((train_split + val_split)*len(listIDs))):]


test_df=data_df[data_df['cropped_geometry_name'].isin(testIDs)]
test_df.to_pickle(os.path.join(path,'test_picklefile.pkl'))

Available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
Memory growth:  True
GPU set to be visible and memory growth set to:  True


In [19]:
if preprocess==True:
    scale = get_scaling_factors(path,'GeometryAll','DoseAll', data_df)
    scale_json = json.dumps(scale)
    with open('scale_mpe.json', 'w') as file:
        file.write(scale_json)

100%|██████████| 34284/34284 [2:10:26<00:00,  4.38it/s]  


In [17]:
test_df

,text_file_names,dose_file_names,energy,beam coordinates,angles,ct_name,cropped_dose_name,cropped_geometry_name,shape
3,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0003,358.677510,"(18.3, -3.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,7e67d860-8861-4766-b82c-ef4b864165ec,13241077-f950-4ec9-8ad2-0b4b5e91a744,"(68, 68, 150)"
5,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0005,163.178216,"(18.3, -1.7, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,7ff14d67-9d52-4b1e-b148-cc8da91a52f3,b03f3230-d549-4c7b-9a8f-a1cc4d0d4cfc,"(68, 68, 150)"
8,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0008,352.737512,"(18.3, 1.3, -6.2)","(0, 90)",2.000000-Abdomen__5.0__B30f-79823,ccb5f960-ca89-4c4c-a8f7-1379f35d0622,c9f2b64c-685c-4769-9865-56f4195ae25d,"(68, 68, 150)"
21,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0021,298.493090,"(-18.3, -4.7, -6.2)","(0, 270)",2.000000-Abdomen__5.0__B30f-79823,69350280-de0b-4680-806c-dc2e2948ac10,23562d3c-13fb-4e9a-be30-d59f7efcfca2,"(68, 68, 150)"
25,2.000000-Abdomen__5.0__B30f-79823.txt,2.000000-Abdomen__5.0__B30f-79823_Run_0025,135.901338,"(-18.3, -0.7, -6.2)","(0, 270)",2.000000-Abdomen__5.0__B30f-79823,cacf9268-61af-4a31-96e7-44a1cba92286,2988df34-1fb2-4a63-97ae-6c8d53208180,"(68, 68, 150)"
...,...,...,...,...,...,...,...,...,...
34299,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2507,310.267861,"(9.2, -16.6, 7.3)","(90, 0)",9.000000-Venous_Phase__5.0__B30f-43947,15cc1ade-23b4-4d6f-b451-523dbd0bd91e,4a7c484a-78c7-4ed7-be1a-87fb70795bd5,"(68, 68, 150)"
34301,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2509,398.214971,"(-8.8, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,7322066f-702a-4039-a166-c4db1d2069f8,ee48c037-1266-4cc0-9cd7-76c3ba8cb72b,"(68, 68, 150)"
34306,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2514,130.799628,"(-3.8, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,a39f4633-cd64-4648-b8a3-d2b812e91d65,bda7c010-bf9f-4e11-a9ea-9d5df7b0be2b,"(68, 68, 150)"
34318,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_2526,355.500052,"(8.2, 16.6, 7.3)","(270, 0)",9.000000-Venous_Phase__5.0__B30f-43947,0d8b364e-7148-4bc9-8826-289cf6ea0d15,bdf8499e-364e-4da6-8fb9-8ebfc89500f4,"(68, 68, 150)"
